In [ ]:
from openai import OpenAI
import json
import os
import random
import tqdm
api_key = ""
client = OpenAI(api_key=api_key)
data_dir = "../../data/pubmed.raw_data.json"
dataset="pubmed"
model="gpt-4o"

with open(data_dir,"r") as f:
    sentences = json.load(f)['original']
if "pubmed" in data_dir:
    questions = [i.split("Answer:")[0] for i in sentences]
    input_sentences = [i.split("Answer: ")[1] for i in sentences]
    print(questions[0])
else:
    input_sentences=sentences

print(len(input_sentences))
print(input_sentences[0])


In [ ]:
res = []
for sentence in tqdm.tqdm(input_sentences):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a professional rewriting expert and you can help paraphrasing this paragraph in English without missing the original details. Please keep the length of the rewritten text similar to the original text."},
            {"role": "user", "content": f"{sentence}"}
        ],
        temperature=0.7
    )
    res.append(completion.choices[0].message.content)

100%|██████████| 150/150 [03:38<00:00,  1.45s/it]


In [ ]:
with open(data_dir,"r") as f:
    sentences = json.load(f)['original']
dic = {
    "original": sentences,
    "rewritten": res
}
if "pubmed" in dataset:
    dic['rewritten'] = [questions[idx] + "Answers: " + j for idx,j in enumerate(dic['rewritten'])]
    
with open(f'../../data/{model}/{dataset}_rewrite_{model}.raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(dic, f, ensure_ascii=False, indent=4)